In [1]:
from datamesh.data_sources.fred import FRED

# Replace 'your_api_key_here' with your actual FRED API key
fred = FRED(api_key='')

# Example: Fetch GDP data
data = fred.fetch_data('GDP')
print(data)

[{'date': '1946-01-01', 'value': '.'}, {'date': '1946-04-01', 'value': '.'}, {'date': '1946-07-01', 'value': '.'}, {'date': '1946-10-01', 'value': '.'}, {'date': '1947-01-01', 'value': '243.164'}, {'date': '1947-04-01', 'value': '245.968'}, {'date': '1947-07-01', 'value': '249.585'}, {'date': '1947-10-01', 'value': '259.745'}, {'date': '1948-01-01', 'value': '265.742'}, {'date': '1948-04-01', 'value': '272.567'}, {'date': '1948-07-01', 'value': '279.196'}, {'date': '1948-10-01', 'value': '280.366'}, {'date': '1949-01-01', 'value': '275.034'}, {'date': '1949-04-01', 'value': '271.351'}, {'date': '1949-07-01', 'value': '272.889'}, {'date': '1949-10-01', 'value': '270.627'}, {'date': '1950-01-01', 'value': '280.828'}, {'date': '1950-04-01', 'value': '290.383'}, {'date': '1950-07-01', 'value': '308.153'}, {'date': '1950-10-01', 'value': '319.945'}, {'date': '1951-01-01', 'value': '336.0'}, {'date': '1951-04-01', 'value': '344.09'}, {'date': '1951-07-01', 'value': '351.385'}, {'date': '1951

In [1]:
from datamesh.data_sources.fred import FredDataSource

fred_data_source = FredDataSource(api_key="")
fetch_response = fred_data_source.fetch_data("GDP")

# Example: Print the fetched data
print(fetch_response.json(indent=2))

{
  "metadata": {
    "source": "FRED",
    "series_id": "GDP"
  },
  "data": [
    {
      "timestamp": "1947-01-01T00:00:00",
      "value": 243.164,
      "notes": "FRED observation"
    },
    {
      "timestamp": "1947-04-01T00:00:00",
      "value": 245.968,
      "notes": "FRED observation"
    },
    {
      "timestamp": "1947-07-01T00:00:00",
      "value": 249.585,
      "notes": "FRED observation"
    },
    {
      "timestamp": "1947-10-01T00:00:00",
      "value": 259.745,
      "notes": "FRED observation"
    },
    {
      "timestamp": "1948-01-01T00:00:00",
      "value": 265.742,
      "notes": "FRED observation"
    },
    {
      "timestamp": "1948-04-01T00:00:00",
      "value": 272.567,
      "notes": "FRED observation"
    },
    {
      "timestamp": "1948-07-01T00:00:00",
      "value": 279.196,
      "notes": "FRED observation"
    },
    {
      "timestamp": "1948-10-01T00:00:00",
      "value": 280.366,
      "notes": "FRED observation"
    },
    {
      "tim

In [3]:
fred_data_source = FredDataSource(mongodb_uri="mongodb://localhost:27017", db_name="market_data")
data = fred_data_source.fetch_data("GDP")

AttributeError: 'FetchResponse' object has no attribute 'items'

In [28]:
import duckdb
import pandas as pd
from typing import List, Optional
from pydantic import BaseModel, Field
import datetime
from models import TimeSeriesData


# Function to simulate upsert behavior
def upsert_data(duckdb_con, data_to_upsert: List[TimeSeriesData]):
    for record in data_to_upsert:
        # Check if the record exists
        exists_query = "SELECT COUNT(*) FROM timeseries_data WHERE timestamp = ?"
        exists = duckdb_con.execute(exists_query, (record.timestamp,)).fetchone()[0] > 0
        
        if exists:
            # Update existing record
            update_query = "UPDATE timeseries_data SET value = ?, notes = ? WHERE timestamp = ?"
            duckdb_con.execute(update_query, (record.value, record.notes, record.timestamp))
        else:
            # Insert new record
            insert_query = "INSERT INTO timeseries_data (timestamp, value, notes) VALUES (?, ?, ?)"
            duckdb_con.execute(insert_query, (record.timestamp, record.value, record.notes))

# Example usage
duckdb_con = duckdb.connect('FRED.duckdb')

# Assuming `data_to_upsert` is populated with TimeSeriesData instances
data_to_upsert = [
    TimeSeriesData(timestamp=datetime.datetime.now(), value=100.0, notes="Example note")
]
upsert_data(duckdb_con, data_to_upsert)

duckdb_con.close()

ConstraintException: Constraint Error: NOT NULL constraint failed: timeseries_data.source

In [1]:
from envoyer import Envoyer

from data_sources.fred import FredDataSource

envoy = Envoyer(mongo_uri="mongodb://localhost:27017", db_name="market_data_db")

fred_data_source = FredDataSource(source_id='FRED', api_key='1bb785afd4dae7403f57b2a5f667cecc')  # Assuming FredDataSource is defined elsewhere
envoy.register_data_source("FRED", fred_data_source)

# Fetch and save data to MongoDB
data = envoy.fetch_data("FRED", save_to_db=False, collection_name="fred_data_collection", series_id="GDP")

data

In [2]:
df = data.to_format('pandas')
df


AttributeError: 'Envoyer' object has no attribute 'fetch_response'

In [6]:
data.fetched_data

FetchResponse(metadata={'source': 'FRED', 'series_id': 'GDP'}, data=[TimeSeriesData(timestamp=datetime.datetime(1947, 1, 1, 0, 0), source='FRED', series_id='GDP', year=1947, month=1, day=1, value=243.164, notes='FRED observation'), TimeSeriesData(timestamp=datetime.datetime(1947, 4, 1, 0, 0), source='FRED', series_id='GDP', year=1947, month=4, day=1, value=245.968, notes='FRED observation'), TimeSeriesData(timestamp=datetime.datetime(1947, 7, 1, 0, 0), source='FRED', series_id='GDP', year=1947, month=7, day=1, value=249.585, notes='FRED observation'), TimeSeriesData(timestamp=datetime.datetime(1947, 10, 1, 0, 0), source='FRED', series_id='GDP', year=1947, month=10, day=1, value=259.745, notes='FRED observation'), TimeSeriesData(timestamp=datetime.datetime(1948, 1, 1, 0, 0), source='FRED', series_id='GDP', year=1948, month=1, day=1, value=265.742, notes='FRED observation'), TimeSeriesData(timestamp=datetime.datetime(1948, 4, 1, 0, 0), source='FRED', series_id='GDP', year=1948, month=4, 

In [10]:

datasource = 'FRED'
series_id = 'GDP'

duckdb_con = duckdb.connect(f'../duckdb/{datasource}.duckdb')

# duckdb_con.sql("INSERT INTO my_table SELECT * FROM df")
duckdb_con.sql(f"CREATE OR Replace TABLE {series_id} AS SELECT * FROM df")

duckdb_con.close()

duckdb_con = duckdb.connect(f'../duckdb/{datasource}.duckdb')
df = duckdb_con.sql(f"SELECT * FROM {series_id}").df()
duckdb_con.close()

df

,timestamp,source,series_id,year,month,day,value,notes
0,1947-01-01,FRED,GDP,1947,1,1,243.164,FRED observation
1,1947-04-01,FRED,GDP,1947,4,1,245.968,FRED observation
2,1947-07-01,FRED,GDP,1947,7,1,249.585,FRED observation
3,1947-10-01,FRED,GDP,1947,10,1,259.745,FRED observation
4,1948-01-01,FRED,GDP,1948,1,1,265.742,FRED observation
...,...,...,...,...,...,...,...,...
303,2022-10-01,FRED,GDP,2022,10,1,26408.405,FRED observation
304,2023-01-01,FRED,GDP,2023,1,1,26813.601,FRED observation
305,2023-04-01,FRED,GDP,2023,4,1,27063.012,FRED observation
306,2023-07-01,FRED,GDP,2023,7,1,27610.128,FRED observation


In [26]:
df = envoy.to_format(data, 'pandas')
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day


In [7]:
conn = duckdb.connect(database=':memory:', read_only=False)

In [8]:
df.to_sql('FRED', con=conn, if_exists='replace', index=False)

/tmp/ipykernel_31064/1463349306.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('FRED', con=conn, if_exists='replace', index=False)


-1

In [1]:
with duckdb.connect('FRED.duckdb') as duckdb_con:
    duckdb_con.execute('''
                CREATE TABLE IF NOT EXISTS timeseries_data (
                timestamp TIMESTAMP,
                source VARCHAR,
                series_id VARCHAR,
                value DOUBLE,
                notes VARCHAR,
                year INTEGER,
                month INTEGER,
                day INTEGER,

                PRIMARY KEY (timestamp, source, series_id))
    ''')


NameError: name 'duckdb' is not defined

In [29]:
data.

AttributeError: 'Envoyer' object has no attribute 'data'

In [17]:
import duckdb
from models import TimeSeriesData
import datetime


# Function to simulate upsert behavior
def upsert_data(duckdb_con, data_to_upsert: list[TimeSeriesData]):
    for record in data_to_upsert:
        # Check if the record exists
        exists_query = "SELECT COUNT(*) FROM timeseries_data WHERE timestamp = ? and source = ? and series_id = ?"
        exists = duckdb_con.execute(exists_query, (record.timestamp, record.source, record.series_id)).fetchone()[0] > 0
        
        if exists:
            # Update existing record
            update_query = "UPDATE timeseries_data SET value = ?, notes = ?, year = ?, month = ?, day = ? WHERE timestamp = ? and source = ? and series_id = ?"
            duckdb_con.execute(update_query, (record.value, record.notes, record.year, record.month, record.day, record.timestamp, record.source, record.series_id))
        else:
            # Insert new record
            insert_query = "INSERT INTO timeseries_data (value, notes, timestamp, year, month, day, source, series_id ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)"
            duckdb_con.execute(insert_query, (record.value, record.notes, record.timestamp, record.year, record.month, record.day, record.source, record.series_id))

# Example usage
duckdb_con = duckdb.connect('FRED.duckdb')

# Assuming `data_to_upsert` is populated with TimeSeriesData instances
data_to_upsert = [
    TimeSeriesData(timestamp=datetime.datetime.today(), year = 2020, month = 12, day = 1, source='FRED', series_id='hi', value=150.0, notes="Example note")
]
upsert_data(duckdb_con, data.fetched_data.data)

duckdb_con.close()

In [18]:
duckdb_con = duckdb.connect('FRED.duckdb')
df = duckdb_con.sql("SELECT * FROM timeseries_data").df()
duckdb_con.close()

df

,timestamp,value,notes,year,month,day,source,series_id
0,1947-01-01,243.164,FRED observation,1947,1,1,FRED,GDP
1,1947-04-01,245.968,FRED observation,1947,4,1,FRED,GDP
2,1947-07-01,249.585,FRED observation,1947,7,1,FRED,GDP
3,1947-10-01,259.745,FRED observation,1947,10,1,FRED,GDP
4,1948-01-01,265.742,FRED observation,1948,1,1,FRED,GDP
...,...,...,...,...,...,...,...,...
303,2022-10-01,26408.405,FRED observation,2022,10,1,FRED,GDP
304,2023-01-01,26813.601,FRED observation,2023,1,1,FRED,GDP
305,2023-04-01,27063.012,FRED observation,2023,4,1,FRED,GDP
306,2023-07-01,27610.128,FRED observation,2023,7,1,FRED,GDP


In [9]:
df

,timestamp,value,notes,year,month,day,source,series_id
0,2024-04-10 18:24:26.539630,100.000,Example note,2020,12,1,FRED,hi
1,2024-04-10 18:24:49.505051,100.000,Example note,2020,12,1,FRED,hi
2,2024-04-10 18:24:59.313515,100.000,Example note,2020,12,1,FRED,hi
3,2024-04-10 18:25:03.158838,100.000,Example note,2020,12,1,FRED,hi
4,2024-04-10 18:25:10.428755,100.000,Example note,2020,12,1,FRED,hi
...,...,...,...,...,...,...,...,...
310,2022-10-01 00:00:00.000000,26408.405,FRED observation,2022,10,1,FRED,GDP
311,2023-01-01 00:00:00.000000,26813.601,FRED observation,2023,1,1,FRED,GDP
312,2023-04-01 00:00:00.000000,27063.012,FRED observation,2023,4,1,FRED,GDP
313,2023-07-01 00:00:00.000000,27610.128,FRED observation,2023,7,1,FRED,GDP


In [ ]:
import duckdb
from models import TimeSeriesData


# # Function to simulate upsert behavior
# def upsert_data(duckdb_con, data_to_upsert: list[TimeSeriesData]):
#     for record in data_to_upsert:
#         # Check if the record exists
#         exists_query = "SELECT COUNT(*) FROM timeseries_data WHERE timestamp = ? and source = ? and series_id = ?"
#         exists = duckdb_con.execute(exists_query, (record.timestamp, record.source, record.series_id)).fetchone()[0] > 0
        
#         if exists:
#             # Update existing record
#             update_query = "UPDATE timeseries_data SET value = ?, notes = ? WHERE timestamp = ? and source = ? and series_id = ?"
#             duckdb_con.execute(update_query, (record.value, record.notes, record.timestamp, record.source, record.series_id))
#         else:
#             # Insert new record
#             insert_query = "INSERT INTO timeseries_data (timestamp, value, notes) VALUES (?, ?, ?)"
#             duckdb_con.execute(insert_query, (record.value, record.notes, record.timestamp, record.source, record.series_id))

# # Example usage
# duckdb_con = duckdb.connect('FRED.duckdb')

# # Assuming `data_to_upsert` is populated with TimeSeriesData instances
# data_to_upsert = [
#     TimeSeriesData(timestamp=datetime.datetime.now(), source='FRED', series_id='hi', value=100.0, notes="Example note")
# ]
# upsert_data(duckdb_con, data_to_upsert)

# duckdb_con.close()

NameError: name 'List' is not defined

In [ ]:
duckdb_con = duckdb.connect('FRED.duckdb')
df = duckdb_con.sql("SELECT * FROM timeseries_data").df()
duckdb_con.close()

In [21]:
duckdb_con = duckdb.connect('FRED.duckdb')

# duckdb_con.sql("INSERT INTO my_table SELECT * FROM df")
duckdb_con.sql("CREATE OR Replace TABLE my_table AS SELECT * FROM df")

duckdb_con.close()

duckdb_con = duckdb.connect('FRED.duckdb')
df = duckdb_con.sql("SELECT * FROM my_table").df()
duckdb_con.close()



In [23]:
duckdb_con = duckdb.connect('FRED.duckdb')
df = duckdb_con.sql("SELECT * FROM my_table").df()
duckdb_con.close()

In [24]:
df

,timestamp,value,notes,source,series_id
0,1947-01-01,243.164,FRED observation,FRED,GDP
1,1947-04-01,245.968,FRED observation,FRED,GDP
2,1947-07-01,249.585,FRED observation,FRED,GDP
3,1947-10-01,259.745,FRED observation,FRED,GDP
4,1948-01-01,265.742,FRED observation,FRED,GDP
...,...,...,...,...,...
303,2022-10-01,26408.405,FRED observation,FRED,GDP
304,2023-01-01,26813.601,FRED observation,FRED,GDP
305,2023-04-01,27063.012,FRED observation,FRED,GDP
306,2023-07-01,27610.128,FRED observation,FRED,GDP


In [10]:
result = conn.execute("SELECT * FROM FRED").fetchdf()
print(result)

     timestamp         value             notes source series_id
0   1947-01-01    243.164001  FRED observation   FRED       GDP
1   1947-04-01    245.968002  FRED observation   FRED       GDP
2   1947-07-01    249.585007  FRED observation   FRED       GDP
3   1947-10-01    259.744995  FRED observation   FRED       GDP
4   1948-01-01    265.742004  FRED observation   FRED       GDP
..         ...           ...               ...    ...       ...
303 2022-10-01  26408.404297  FRED observation   FRED       GDP
304 2023-01-01  26813.601562  FRED observation   FRED       GDP
305 2023-04-01  27063.011719  FRED observation   FRED       GDP
306 2023-07-01  27610.128906  FRED observation   FRED       GDP
307 2023-10-01  27956.998047  FRED observation   FRED       GDP

[308 rows x 5 columns]


In [14]:
market_data_interface.data_sources.keys()

dict_keys(['FRED'])